In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [2]:
import pymysql as my

class DBHelper:
    '''
    맴버변수 : 커넥션 
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
    '''
    맴버 함수
    '''
    def db_init(self):
        self.conn = my.connect(
                        host='localhost',
                        user='root',
                        password='054867',
                        db='pythondb',
                        charset='utf8',
                        cursorclass=my.cursors.DictCursor )
    
    def db_free(self):
        if self.conn:
            self.conn.close()

    # 검색 키워드 가져오기 => 웹에서 검색
    def db_selectKeyword(self):
        # 커서 오픈
        # with => 닫기를 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql  = "select * from tbl_keyword;"
            cursor.execute(sql)
            rows = cursor.fetchall()
            print(rows)
        return rows
        
    def db_insertCrawlingData(self, title, price, area, contents, keyword ):
        with self.conn.cursor() as cursor:
            sql = "insert into tbl_crawlingdata (title, price, area, contents, keyword) values( %s,%s,%s,%s,%s );"
            cursor.execute(sql, (title, price, area, contents, keyword) )
        self.conn.commit()

In [3]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    area  = ''
    link  = ''
    img   = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, area, link, img, contents=None ):
        self.title = title
        self.price = price
        self.area  = area
        self.link  = link
        self.img   = img
        self.contents = contents

In [4]:
# 사전에 필요한 정보를 로드 => 디비혹스 쉘, 베치 파일에서 인자로 받아서 세팅
db=DBHelper()
main_url = 'http://tour.interpark.com/' 
keyword  = '로마'
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'db' is deprecated, use 'database'
  if sys.path[0] == '':


In [5]:
# 윈도우용
driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')

C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [6]:
# 사이트 접속 (get)
driver.get(main_url)

In [7]:
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)

C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\selenium\webdriver\remote\webdriver.py:390: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [8]:
# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')
# 검색 버튼 클릭
time.sleep(2)
driver.find_element_by_css_selector('button.search-btn').click()

C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\selenium\webdriver\remote\webdriver.py:670: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [9]:
# driver.find_element_by_css_selector('button.recent-btn').click()

In [9]:
# 잠시 대기 => 페이가 로드되고 나서 즉각적으로 데이터를 획득 하는 행위는 
# 명시적 대기 => 특정 요소가 로케이트(발결된때까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        # 지정한 한개 요소가 올라면 웨이트 종료
        EC.presence_of_element_located( (By.CLASS_NAME, 'oTravelBox') )
    )
except Exception as e:
    print( '오류 발생', e)
# 암묵적 대기 => DOM이 다 로드 될때까지 대기 하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간 동안 DOM 풀링을 지시 예를 들어 10 초이내 라로 
# 발견 되면 진행
driver.implicitly_wait( 10 )
# 절대기 대기 => time.sleep(10) -> 클라우드 페어(디도스 방어  솔류션)
# 더보기 눌러서 => 게시판 진입 

In [10]:
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

In [11]:
# searchModule.SetCategoryList(1, '') 스크립트 실행
# 16은 임시값, 게시물을 넘어갔을때 현상을 확인차
for page in range(2, 3): # 2):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        #############################################################
        # 여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            # 이미지를 링크값을 사용할것인가? 
            # 직접 다운로드 해서 우리 서버에 업로드(ftp) 할것인가?
            print( '썸네임', li.find_element_by_css_selector('img').get_attribute('src') )
            print( '링크', li.find_element_by_css_selector('a').get_attribute('onclick') )
            print( '상품명', li.find_element_by_css_selector('h5.proTit').text )
            print( '코멘트', li.find_element_by_css_selector('.proSub').text )
            print( '가격',   li.find_element_by_css_selector('.proPrice').text )
            area = ''
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print(  info.text )
            print('='*100)
            # 데이터 모음
            # li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            # 데이터가 부족하거나 없을수도 있으므로 직접 인덱스로 표현은 위험성이 있음
            obj = TourInfo(  
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append( obj )
    except Exception as e1:
        print( '오류', e1 )

print( tour_list, len(tour_list) )


2 페이지 이동
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3015110_2_920.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3015110','')
상품명 [서유럽 10국 16일] 하나도빠지지않는 완전일주+바티칸하이패스+뮈렌


C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\selenium\webdriver\remote\webdriver.py:689: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\selenium\webdriver\remote\webelement.py:487: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


코멘트 서유럽 완전정복! 베네룩스+남프랑스+이태리 일주까지!
가격 3,090,000 원~
여행 기간 : 14박16일
출발 가능 기간 : 2021.07.02~2021.12.31

C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\selenium\webdriver\remote\webelement.py:505: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")



평점 8.8
56개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3015084_14_480.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3015084','')
상품명 [서유럽 4국 12일] 1급UP! 모나코+융프라우 산악열차
코멘트 루체른/베른 + 모나코/니스
가격 3,540,000 원~
여행 기간 : 10박12일
출발 가능 기간 : 2021.07.01~2021.12.30
평점 9.5
48개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3015996_7_217.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3015996','')
상품명 [서유럽4국10일]런던+옥스포드/윈저 내부관람+리기정상식
코멘트 영국2DAY 로 깊이있게
가격 2,050,000 원~
여행 기간 : 8박10일
출발 가능 기간 : 2021.07.01~2021.12.30
평점 9.2
15개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3013164_2_283.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3013164','')
상품명 [♥프랑스/스위스/이탈리아/오스트리아/독일 10일]
코멘트 ■ 실속 서유럽 5국 핵심도시!
가격 2,150,000 원~
여행 기간 : 8박10일
출발 가능 기간 : 2021.07.02~2021.12.31
평점 9.6
13개의 상품평
썸네임 http://tourimage.interpark.com/product/t

In [12]:
for tour in tour_list:
    # tour => TourInfo
    print( type(tour) )
    # 링크 데이터에서 실데이터 획득
    # 분해
    arr = tour.link.split(',')
    if arr:
        # 대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        # 슬라이싱 => 앞에 ', 뒤에 ' 제거
        detail_url = link[1:-1]
        # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
        driver.get( detail_url )
        time.sleep(2)
        # pip install bs4
        # 혖재 페이지를 beautifulsoup 의 DOM으로 구성
        soup = bs( driver.page_source, 'html.parser')
        # 현제 상세 정보 페이지에서 스케줄 정보 획득
        data = soup.select('.tip-cover')
        print( type(data), len(data), type(data[0].contents)  )
        # 
        # 데이터 sum
        data[0].contents = 'test' # me
        content_final = ''
        for c in data[0].contents:
            content_final += str(c)
        
        # html 콘첸츠 데이터 전처리 (디비에 입력 가능토록)
        import re
        content_final   = re.sub("'", '"', content_final)
        content_final   = re.sub(re.compile(r'\r\n|\r|\n|\n\r+'), '', content_final)

        print( content_final )
        # 콘텐츠 내용에 따라 전처리 => data[0].contents
        db.db_insertCrawlingData(
            tour.title,
            tour.price[:-1],
            tour.area.replace('출발 가능 기간 : ',''),
            content_final,
            keyword
        )

<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test


In [13]:
driver.close()

In [13]:
# import sys 

# sys.exit()

In [14]:
# DB test        
if __name__=='__main__':
    db = DBHelper()
    print( db.db_selectKeyword() )
    print( db.db_insertCrawlingData('1','2','3','4','5') )
    db.db_free()

()
()
None


C:\Users\SEC\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'db' is deprecated, use 'database'
  if sys.path[0] == '':


### data

In [18]:
tour_list[0].area

'출발 가능 기간 : 2021.07.01~2021.12.30'

In [21]:
arr=tour_list[0].link.split(',')

In [22]:
len(arr)

2

In [26]:
link=arr[0].replace("searchModule.OnClickDetail('",'')

In [28]:
detail_url=link[0:-1]

In [30]:
detail_url

'http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3015008'

In [33]:
driver.get(detail_url)

In [36]:
soup=bs(driver.page_source,'html.parser')

In [37]:
data=soup.select('.tip-cover')

In [38]:
driver.close()

In [39]:
print('hello! \n')
print('hello!\r')
print('hello!\r\n')

hello! 

hello!
hello!



In [46]:
# contend_final = data[0].contents

In [41]:
# import re

In [47]:
# contend_final = re.sub("''",'""',contend_final)
# contend_final = re.sub(re.compile(r'\r\n|\r|\n|\n\r+'),'',contend_final)

TypeError: expected string or bytes-like object

In [ ]:
# for tour in tour_list:DD
#     # tour => TourInfo
#     print( type(tour) )
#     # 링크 데이터에서 실데이터 획득
#     # 분해
#     arr = tour.link.split(',')
#     if arr:
#         # 대체
#         link = arr[0].replace('searchModule.OnClickDetail(','')
#         # 슬라이싱 => 앞에 ', 뒤에 ' 제거
#         detail_url = link[1:-1]
#         # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
#         driver.get( detail_url )
#         time.sleep(2)
#         # pip install bs4
#         # 혖재 페이지를 beautifulsoup 의 DOM으로 구성
#         soup = bs( driver.page_source, 'html.parser')
#         # 현제 상세 정보 페이지에서 스케줄 정보 획득
#         data = soup.select('.tip-cover')
#         print( type(data), len(data), type(data[0].contents)  )
#         # 
#         # 데이터 sum
#         data[0].contents = 'test' # me
#         content_final = ''
#         for c in data[0].contents:
#             content_final += str(c)
        
#         # html 콘첸츠 데이터 전처리 (디비에 입력 가능토록)
#         import re
#         content_final   = re.sub("'", '"', content_final)
#         content_final   = re.sub(re.compile(r'\r\n|\r|\n|\n\r+'), '', content_final)

#         print( content_final )
#         # 콘텐츠 내용에 따라 전처리 => data[0].contents
#         db.db_insertCrawlingData(
#             tour.title,
#             tour.price[:-1],
#             tour.area.replace('출발 가능 기간 : ',''),
#             content_final,
#             keyword
#         )